In [ ]:
import os
from pathlib import Path
from lxml import etree
import shutil

In [ ]:
all_files = [os.path.join('ARCH_MSS_UNICODE-utf8_dir', file) for file in os.listdir('ARCH_MSS_UNICODE-utf8_dir')]
all_files = sorted(all_files, key=lambda x:int(os.path.splitext(os.path.split(x)[1])[0]))

In [ ]:
for file in all_files:
    tree = etree.parse(file)
    file_num = os.path.splitext(os.path.split(file)[1])[0]
    for i in tree.xpath("//datafield[@tag='999']/subfield[@code='a']"):
        main_files.add(os.path.getsize(file))
        output_dir = os.path.join("ARCH_MSS_UNICODE-utf8_bunched", file_num)
        os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, '{}.xml'.format(file_num))
    shutil.copy2(file, output_path)

In [ ]:
all_xmls = [os.path.join(root, file)
            for root, dirs, files in os.walk('ARCH_MSS_UNICODE-utf8_bunched')
            for file in files
            if os.path.splitext(file)[1] == '.xml'
           ]
sorted_xmls = sorted(all_xmls, key=lambda x: int(os.path.split(os.path.split(x)[0])[1]))
parent_dirs = set(os.path.split(x)[0] for x in all_xmls)

In [ ]:
for parent_dir in sorted(parent_dirs):
    all_999s = set()
    for file in os.listdir(parent_dir):
        full_path = os.path.join(parent_dir, file)
        tree = etree.parse(full_path)
        try:
            title = tree.xpath('//datafield[@tag="245"]/subfield[@code="a"]')[0].text
        except IndexError:
            title = ''
        for i in tree.xpath("//datafield[@tag='999']/subfield[@code='a']"):
            all_999s.add(i.text)
    for file in os.listdir(parent_dir):
        full_path = os.path.join(parent_dir, file)
        tree = etree.parse(full_path)
        for field in tree.xpath("//datafield[@tag='852']"):
            h_subfields = field.xpath('//subfield[@code="h"]')
            i_subfields = field.xpath('//subfield[@code="i"]')
            if h_subfields and i_subfields:
                concat_fields = '{} {}'.format(h_subfields[0].text, i_subfields[0].text).strip()
            elif h_subfields and not i_subfields:
                for i in h_subfields:
                    if i.text:
                        concat_fields = i.text.strip()
                        break
            else:
                print('{} has no h or i subfields'.format(file))
                concat_files = ''
                break
#             if len(field.xpath('//subfield[@code="h"]')) > 1 or len(field.xpath('//subfield[@code="i"]')) > 1:
#                 print(file)
            match = False
            for a_999 in all_999s:
                if concat_fields.replace(' ', '').replace('.', '').replace(',','').replace(':','').lower() in a_999.replace(' ', '').replace('.', '').replace(',','').replace(':','').lower():
                    match = True
            if not match:
                print(file, '\t', title, '\t', concat_fields, '\t', all_999s)

                